# Using EcoFOCIpy to process raw field data

## DY1908 (Fall Mooring Cruise - Dyson)

## CTD / BTL Data

Basic workflow for each instrument grouping is *(initial archive level)*:
- SBE workflow must happen first
- Parse data from btl files into pandas dataframe

Convert to xarray dataframe for all following work *(working or final data level):
- Add metadata from cruise yaml files and/or header info
- ingest metadata from cruise / cast logs
- process data beyond simple file translate
- apply any calibrations or corrections
    + field corrections
    + offsets
    + instrument compensations
    + some QC were available... this would be old-school simple bounds mostly
- adjust time bounds and sample frequency (xarray dataframe)
- save as CF netcdf via xarray: so many of the steps above are optional
    + **ERDDAP NRT** if no corrections, offsets or time bounds are applied but some meta data is
    + **Working and awaiting QC** has no ERDDAP representation and is a holding spot
    + **ERDDAP Final** fully calibrated, qc'd and populated with meta information

Plot for preview and QC
- preview images (indiv and/or collectively)
- manual qc process
- automated qc process ML/AI

Further refinenments for ERDDAP hosting:

## Differences from EPIC standard
- previously btl files had coordinates of lat/lon/time/depth...with a bottle position / fireing order being a variable, this means that if multiple bottles are fired at the same depth, the file was not uniquely indexed and the bottle variable had to be conflated for multiple fireings
- in new version, we will index based on lat/lon/time/bottle_num (bottle number is a sequential unique value... often representing the position on the rosette), merging with CTD downcast data will require maintining a pressure/depth variable in the bottle data that can be rounded to the nearest 1m bin.  This does not solve the problem with multiple discrete samples taken at a single depth and single niskin though.

## Example below is for SBE 9/11+ V2 but the workflow is similar for any SBE instruments.

Future processing of this instrument can be a simplified (no markdown) process which can be archived so that the procedure can be traced or updated

We process each cast as an individual file so this example will not loop over all profiles.  See `example/all_casts.py` example for processing an entire cruise at once.

Adding Discrete samples such as Oxygen, Chlorophyll, Salinity to BTL Data is in `example/discrete_castdata.py`.  Its purpose is to match niskin/bottle information to depth for the discrete data.

In [2]:
import yaml
import glob

import EcoFOCIpy.io.sbe_ctd_parser as sbe_ctd_parser #<- instrument specific
import EcoFOCIpy.io.ncCFsave as ncCFsave
import EcoFOCIpy.metaconfig.load_config as load_config

The sample_data_dir should be included in the github package but may not be included in the pip install of the package

## Simple Processing - first step

In [3]:
sample_data_dir = '/Users/bell/ecoraid/2019/CTDcasts/dy1908/' #root path to cruise directory
ecofocipy_dir = '/Users/bell/Programs/EcoFOCIpy/'

In [4]:
###############################################################
# edit to point to {cruise sepcific} raw datafiles 
datafile = sample_data_dir+'rawconverted/' #<- point to cruise and process all files within
cruise_name = 'DY1908' #no hyphens
cruise_meta_file = sample_data_dir+'logs/DY1908.yaml'
inst_meta_file = sample_data_dir+'logs/FOCI_standard_CTD.yaml' #<- copy to each deployment for simplicity?
group_meta_file = ecofocipy_dir+'staticdata/institutional_meta_example.yaml'
###############################################################

#init and load data
cruise = sbe_ctd_parser.sbe_btl()
filename_list = sorted(glob.glob(datafile + '*.btl'))

cruise_data = cruise.manual_parse(filename_list)

Processing /Users/bell/ecoraid/2019/CTDcasts/dy1908/rawconverted/ctd001.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1908/rawconverted/ctd002.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1908/rawconverted/ctd003.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1908/rawconverted/ctd004.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1908/rawconverted/ctd005.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1908/rawconverted/ctd006.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1908/rawconverted/ctd007.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1908/rawconverted/ctd008.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1908/rawconverted/ctd009.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1908/rawconverted/ctd010.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1908/rawconverted/ctd011.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1908/rawconverted/ctd012.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1908/rawconverted/ctd013.btl
Processing /

/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

Processing /Users/bell/ecoraid/2019/CTDcasts/dy1908/rawconverted/ctd019.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1908/rawconverted/ctd020.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1908/rawconverted/ctd021.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1908/rawconverted/ctd022.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1908/rawconverted/ctd023.btl
Processing /Users/bell/ecoraid/2019/CTDcasts/dy1908/rawconverted/ctd024.btl


/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

In [5]:
#quick statistical look at the distribution of data for a cast
# #preview a dataframe
cruise_data['ctd001.btl'].describe()

,sbeox0ml/l,sbeox0ps,sbox0mm/kg,sbeox1ml/l,sbeox1ps,sbox1mm/kg,sal00,sal11,sigma-t00,sigma-t11,fleco-afl,t090c,t190c,turbwetntu0,prdm,scan,datetime
count,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8
mean,5.579225,85.540625,243.224625,5.620688,86.178125,245.032125,31.754200,31.751600,24.490800,24.488663,0.987413,9.143775,9.144525,0.344175,28.148500,8843.875000,2019-09-20 10:11:49.874999808
min,4.934800,67.803000,214.956000,4.972500,68.321000,216.597000,31.650300,31.647600,24.014300,24.012800,0.131700,4.666600,4.667600,0.202500,1.559000,4800.000000,2019-09-20 10:09:01
25%,4.940075,67.879250,215.185250,4.973850,68.343500,216.656750,31.652050,31.649675,24.015950,24.014100,0.143800,4.669100,4.669600,0.232025,8.058750,7381.500000,2019-09-20 10:10:49
50%,5.874750,94.625500,256.209000,5.915000,95.274000,257.965500,31.653100,31.650750,24.031000,24.028450,1.229750,11.773550,11.776450,0.243800,25.281500,9094.500000,2019-09-20 10:12:00.500000
75%,6.018275,97.099500,262.473250,6.061975,97.803250,264.377500,31.924325,31.921500,25.272850,25.270625,1.634650,11.859875,11.859500,0.513875,43.085250,10775.250000,2019-09-20 10:13:10.249999872
max,6.027300,97.232000,262.865000,6.084100,98.151000,265.342000,31.924600,31.921600,25.273300,25.270900,1.711700,11.876000,11.874700,0.556100,69.862000,11533.000000,2019-09-20 10:13:42
std,0.536070,14.706349,23.514067,0.542414,14.846838,23.791559,0.140970,0.140723,0.647831,0.647689,0.753329,3.706766,3.706802,0.159264,24.521975,2399.445855,NaN


## Create BTL report file

In [6]:
# btl report file
for cast in cruise_data.keys():
    try:
        df=cruise_data[cast]
        df['cast'] = cast.lower().split('.')[0]
        if cast.lower().split('.')[0] == 'ctd001':
            df.to_csv(f'{cruise_name}.report_btl')
        else:
            df.to_csv(f'{cruise_name}.report_btl',mode='a', header=False)
    except:
        print(f'some issue in {cast}')

## Time Properties

Not traditionally dealt with for CTD files as they are likely dynamically updated via GPS feed.  However, FOCI tends to label the date/time with the ***at depth*** time-stamp

## Depth Properties and other assumptions

- currently, all processing and binning (1m for FOCI) is done via seabird routines and the windows software.  This may change with the python ctd package for a few tasks

## Add Deployment meta information

In [7]:
#just a dictionary of dictionaries - simple
with open(cruise_meta_file) as file:
    cruise_config = yaml.full_load(file)
cruise_config[cruise_name]

{'CruiseID': 'DY1908',
 'CruiseID_Historic': '',
 'CruiseID_Alternates': '',
 'Project_Leg': '',
 'Vessel': 'NOAAS Oscar Dyson',
 'ShipID': 'DY',
 'StartDate': datetime.date(2019, 9, 18),
 'EndDate': datetime.date(2019, 10, 2),
 'Project': 'EcoFOCI',
 'ChiefScientist': 'Peter Proctor',
 'StartPort': 'Kodiak, AK',
 'EndPort': 'Dutch Harbor, AK',
 'CruiseLocation': 'Bering Sea',
 'Description': 'FOCI Fall Mooring Survey',
 'CruiseYear': 2019,
 'ctdlogs_pdf_name': 'DY1908_CastLogs.pdf'}

In [8]:
#and if you want a cast from the cruise, just use the consective cast number
cruise_config['CTDCasts']['CTD001']

{'id': 24192,
 'Vessel': 'R/V Oscar Dyson',
 'CruiseID': 'DY1908',
 'Project_Leg': '',
 'UniqueCruiseID': 'DY1908',
 'Project': 'EcoFOCI Fall Mooring Cruise',
 'StationNo_altname': '1',
 'ConsecutiveCastNo': 'CTD001',
 'LatitudeDeg': 56,
 'LatitudeMin': 45.98,
 'LongitudeDeg': 164,
 'LongitudeMin': 20.44,
 'GMTDay': 20,
 'GMTMonth': 'Sep',
 'GMTYear': 2019,
 'GMTTime': 36540,
 'DryBulb': 10.7,
 'RelativeHumidity': 69,
 'WetBulb': -99.9,
 'Pressure': 1002,
 'SeaState': '-99',
 'Visibility': '-99',
 'WindDir': 313,
 'WindSpd': 14.0,
 'CloudAmt': '-99',
 'CloudType': '-99',
 'Weather': '-99',
 'SurfaceTemp': -99.9,
 'BottomDepth': 74,
 'StationNameID': 'M2W',
 'MaxDepth': 69,
 'InstrumentSerialNos': 'Press SN =0772, Pri Temp SN = 4379, Sec Temp SN =2376, Pri Cond SN = 2985, Sec Cond Sn =3127, PAR Sn =70547, Fluor Sn = FLNTU-2057, pri O2 Sn = 1961, sec O2 Sn = 3200 ',
 'Notes': '',
 'NutrientBtlNiskinNo': '1;2;3;4;5;6;7;8',
 'NutrientBtlNumbers': ';499;500;501;502;503;504;505;',
 'OxygenBt

## Add Instrument meta information

Time, depth, lat, lon should be added regardless (always our coordinates) but for a mooring site its going to be a (1,1,1,t) dataset
The variables of interest should be read from the data file and matched to a key for naming.  That key is in the inst_config file seen below and should represent common conversion names in the raw data

In [10]:
with open(inst_meta_file) as file:
    inst_config = yaml.full_load(file)
inst_config

{'time': {'epic_key': 'TIM_601',
  'name': 'time',
  'generic_name': 'time',
  'standard_name': 'time',
  'long_name': 'date and time since reference time'},
 'depth': {'epic_key': 'D_3',
  'generic_name': 'depth',
  'units': 'meter',
  'long_name': 'depth below surface (meters)',
  'standard_name': 'depth',
  'positive': 'down'},
 'latitude': {'epic_key': 'LON_501',
  'name': 'latitude',
  'generic_name': 'latitude',
  'units': 'degrees_north',
  'long_name': 'latitude',
  'standard_name': 'latitude'},
 'longitude': {'epic_key': 'LAT_500',
  'name': 'longitude',
  'generic_name': 'longitude',
  'units': 'degrees_east',
  'long_name': 'longitude',
  'standard_name': 'longitude'},
 'temperature_ch1': {'epic_key': 'T_28',
  'generic_name': 'temp channel 1',
  'long_name': 'Sea temperature in-situ ITS-90 scale',
  'standard_name': 'sea_water_temperature',
  'units': 'degree_C'},
 'temperature_ch2': {'epic_key': 'T2_35',
  'generic_name': 'temp channel 2',
  'long_name': 'Sea temperature i

In [11]:
#sbe data uses header info to name variables... but we want standard names from the dictionary I've created, so we need to rename column variables appropriately
#rename values to appropriate names, if a value isn't in the .yaml file, you can add it

#*** biggest *** difference between moored and profile data is there may be multiple instruments with the same dataype (e.g.) temperature
# on the same platform.  We _used_ to use the phrases primary and secondary, but will now only refer to them as ch1, ch2 etc
cruise_data['ctd001.btl'] = cruise_data['ctd001.btl'].rename(columns={
                        't090c':'temperature_ch1',
                        't190c':'temperature_ch2',
                        'sal00':'salinity_ch1',
                        'sal11':'salinity_ch2',
                        'sbox0mm/kg':'oxy_conc_ch1',
                        'sbeox0ml/l':'oxy_concM_ch1',
                        'sbox1mm/kg':'oxy_conc_ch2',
                        'sbeox1ml/l':'oxy_concM_ch2',
                        'sbeox0ps':'oxy_percentsat_ch1',
                        'sbeox1ps':'oxy_percentsat_ch2',
                        'sigma-t00':'sigma_t_ch1',
                        'sigma-t11':'sigma_t_ch2',
                        'cstarat0':'Attenuation',
                        'cstartr0':'Transmittance',
                        'fleco-afl':'chlor_fluorescence',
                        'turbwetntu0':'turbidity',
                        'empty':'empty', #this will be ignored
                        'prdm':'Pressure [dbar]',
                        'flag':'flag'})

cruise_data['ctd001.btl'].sample()

,oxy_concM_ch1,oxy_percentsat_ch1,oxy_conc_ch1,oxy_concM_ch2,oxy_percentsat_ch2,oxy_conc_ch2,salinity_ch1,salinity_ch2,sigma_t_ch1,sigma_t_ch2,chlor_fluorescence,temperature_ch1,temperature_ch2,turbidity,Pressure [dbar],scan,datetime,cast
bottle,,,,,,,,,,,,,,,,,,
2.0,4.94,67.877,215.183,4.9725,68.324,216.599,31.9243,31.9215,25.273,25.2707,0.1375,4.6679,4.6684,0.5456,50.946,6594.0,2019-09-20 10:10:16,ctd001


In [12]:
cruise_data['ctd001.btl'].columns

Index(['oxy_concM_ch1', 'oxy_percentsat_ch1', 'oxy_conc_ch1', 'oxy_concM_ch2',
       'oxy_percentsat_ch2', 'oxy_conc_ch2', 'salinity_ch1', 'salinity_ch2',
       'sigma_t_ch1', 'sigma_t_ch2', 'chlor_fluorescence', 'temperature_ch1',
       'temperature_ch2', 'turbidity', 'Pressure [dbar]', 'scan', 'datetime',
       'cast'],
      dtype='object')

## Add institutional meta-information


In [13]:
with open(group_meta_file) as file:
    group_config = yaml.full_load(file)
group_config

{'source_documents': 'http://www.oceansites.org/docs/oceansites_data_format_reference_manual.pdf',
 'institution': 'Pacific Marine Environmental Lab (PMEL)',
 'project': 'EcoFOCI',
 'project_url': 'https://www.ecofoci.noaa.gov',
 'principal_investigator': 'Phyllis Stabeno',
 'principal_investigator_email': 'phyllis.stabeno (at) noaa.gov',
 'creator_name': 'Shaun Bell',
 'creator_email': 'shaun.bell (at) noaa.gov',
 'creator_institution': 'PMEL',
 'keywords': 'Mooring, Oceanographic',
 'comment': 'Provisional data',
 'sea_area': 'Bering Sea (BS)',
 'featureType': 'timeSeries',
 'conventions': '”CF-1.6, ~OceanSITES-1.5, ACDD-1.2”',
 'license': '',
 'references': '',
 'citation': '',
 'acknowledgement': ''}

In [14]:
# Add meta data and prelim processing based on meta data
# Convert to xarray and add meta information - save as CF netcdf file
# pass -> data, instmeta, depmeta
cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data['ctd001.btl'], 
                                instrument_yaml=inst_config, 
                                operation_yaml=cruise_config,
                                operation_type='ctd')
cruise_data_nc

In [15]:
cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 8)
Coordinates:
  * bottle              (bottle) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0
Data variables: (12/18)
    oxy_concM_ch1       (bottle) float64 4.935 4.94 4.94 ... 6.016 6.027 6.027
    oxy_percentsat_ch1  (bottle) float64 67.8 67.88 67.88 ... 97.06 97.23 97.22
    oxy_conc_ch1        (bottle) float64 215.0 215.2 215.2 ... 262.4 262.9 262.8
    oxy_concM_ch2       (bottle) float64 4.973 4.973 4.974 ... 6.058 6.074 6.084
    oxy_percentsat_ch2  (bottle) float64 68.32 68.32 68.35 ... 97.74 97.99 98.15
    oxy_conc_ch2        (bottle) float64 216.6 216.6 216.7 ... 264.2 264.9 265.3
    ...                  ...
    temperature_ch2     (bottle) float64 4.668 4.668 4.67 ... 11.86 11.86 11.86
    turbidity           (bottle) float64 0.5561 0.5456 0.5033 ... 0.2442 0.2434
    Pressure [dbar]     (bottle) float64 69.86 50.95 40.47 ... 10.22 1.572 1.559
    scan                (bottle) float64 4.8e+03 6.594e+03 ... 1.153e+04
    datetime            (bottle) datetime64[ns] 2019-09-20T10:09:01 ... 2019-...
    cast                (bottle) object 'ctd001' 'ctd001' ... 'ctd001' 'ctd001'

At this point, you could save your file with the `.xarray2netcdf_save()` method and have a functioning dataset.... but it would be very simple with no additional qc, meta-data, or tuned parameters for optimizing software like ferret or erddap.

In [16]:
# expand the dimensions and coordinate variables
# renames them appropriatley and prepares them for meta-filled values
cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

In [17]:
#build list from columsn in data - if a variable isn't in the yaml file, it will be dropped from the final data fields
cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data['ctd001.btl'].columns.values),drop_missing=False)
#adding dimension meta needs to come after updating the dimension values... BUG?
cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])

The following steps can happen in just about any order and are all meta-data driven.  Therefore, they are not required to have a functioning dataset, but they are required to have a well described dataset

In [18]:
cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 8, latitude: 1, longitude: 1, time: 1)
Coordinates:
  * bottle              (bottle) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0
  * latitude            (latitude) float64 1e+35
  * longitude           (longitude) float64 1e+35
  * time                (time) float64 1e+35
Data variables: (12/18)
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 4.935 ......
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 67.8 ... ...
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 215.0 ......
    oxy_concM_ch2       (latitude, longitude, time, bottle) float64 4.973 ......
    oxy_percentsat_ch2  (latitude, longitude, time, bottle) float64 68.32 ......
    oxy_conc_ch2        (latitude, longitude, time, bottle) float64 216.6 ......
    ...                  ...
    temperature_ch2     (latitude, longitude, time, bottle) float64 4.668 ......
    turbidity           (latitude, longitude, time, bottle) float64 0.5561 .....
    Pressure [dbar]     (latitude, longitude, time, bottle) float64 69.86 ......
    scan                (latitude, longitude, time, bottle) float64 4.8e+03 ....
    datetime            (latitude, longitude, time, bottle) datetime64[ns] 20...
    cast                (latitude, longitude, time, bottle) object 'ctd001' ....

In [19]:
#add global attributes
cruise_data_nc.deployment_meta_add(conscastno='CTD001')

#add instituitonal global attributes
cruise_data_nc.institution_meta_add(group_config)

#add creation date/time - provenance data
cruise_data_nc.provinance_meta_add()

#provide intial qc status field
cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 8, latitude: 1, longitude: 1, time: 1)
Coordinates:
  * bottle              (bottle) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0
  * latitude            (latitude) float64 1e+35
  * longitude           (longitude) float64 1e+35
  * time                (time) float64 1e+35
Data variables: (12/18)
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 4.935 ......
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 67.8 ... ...
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 215.0 ......
    oxy_concM_ch2       (latitude, longitude, time, bottle) float64 4.973 ......
    oxy_percentsat_ch2  (latitude, longitude, time, bottle) float64 68.32 ......
    oxy_conc_ch2        (latitude, longitude, time, bottle) float64 216.6 ......
    ...                  ...
    temperature_ch2     (latitude, longitude, time, bottle) float64 4.668 ......
    turbidity           (latitude, longitude, time, bottle) float64 0.5561 .....
    Pressure [dbar]     (latitude, longitude, time, bottle) float64 69.86 ......
    scan                (latitude, longitude, time, bottle) float64 4.8e+03 ....
    datetime            (latitude, longitude, time, bottle) datetime64[ns] 20...
    cast                (latitude, longitude, time, bottle) object 'ctd001' ....
Attributes: (12/44)
    CruiseID:                      DY1908
    CruiseID_Historic:             
    CruiseID_Alternates:           
    Project_Leg:                   
    Vessel:                        NOAAS Oscar Dyson
    ShipID:                        DY
    ...                            ...
    references:                    
    citation:                      
    acknowledgement:               
    date_created:                  2024-05-24T07:37:26Z
    date_modified:                 
    QC_indicator:                  unqcd

## Save CF Netcdf files

Currently stick to netcdf3 classic... but migrating to netcdf4 (default) may be no problems for most modern purposes.  Its easy enough to pass the `format` kwargs through to the netcdf api of xarray.

In [20]:
#loop over all casts and perform tasks shown above

for cast in cruise_data.keys():
    try:
        cruise_data[cast] = cruise_data[cast].rename(columns={
                            't090c':'temperature_ch1',
                            't190c':'temperature_ch2',
                            'sal00':'salinity_ch1',
                            'sal11':'salinity_ch2',
                            'sbox0mm/kg':'oxy_conc_ch1',
                            'sbeox0ml/l':'oxy_concM_ch1',
                            'sbox1mm/kg':'oxy_conc_ch2',
                            'sbeox1ml/l':'oxy_concM_ch2',
                            'sbeox0ps':'oxy_percentsat_ch1',
                            'sbeox1ps':'oxy_percentsat_ch2',
                            'sigma-t00':'sigma_t_ch1',
                            'sigma-t11':'sigma_t_ch2',
                            'cstarat0':'Attenuation',
                            'cstartr0':'Transmittance',
                            'fleco-afl':'chlor_fluorescence',
                            'turbwetntu0':'turbidity',
                            'prdm':'pressure',
                            'empty':'empty', #this will be ignored
                            'flag':'flag'})

        cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data[cast], 
                                    instrument_yaml=inst_config, 
                                    operation_yaml=cruise_config,
                                    operation_type='ctd')

        cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

        cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data[cast].columns.values),drop_missing=False)
        #adding dimension meta needs to come after updating the dimension values... BUG?
        cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])
        cruise_data_nc.temporal_geospatioal_meta_data_ctd(positiveE=False,conscastno=cast.split('.')[0])

        #add global attributes
        cruise_data_nc.deployment_meta_add(conscastno=cast.split('.')[0].upper())

        #add instituitonal global attributes
        cruise_data_nc.institution_meta_add(group_config)

        #add creation date/time - provenance data
        cruise_data_nc.provinance_meta_add()

        #provide intial qc status field
        cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

        cast_label = cast.lower().split('d')[-1].split('.')[0]
        cruise_data_nc.xarray2netcdf_save(xdf = cruise_data_nc.get_xdf(),
                                   filename=cruise_name+'c'+cast_label.zfill(3)+'_btl.nc',format="NETCDF3_CLASSIC")

  
    except:
        print(f'Skipping {cast}')

/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncCFsave.py:320: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 1900-01-01'. Resolution of 'minutes' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  xdf.to_netcdf(filename,format=kwargs['format'],encoding={'time':{'units':'days since 1900-01-01'}})
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncCFsave.py:320: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 1900-01-01'. Resolution of 'minutes' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  xdf.to_netcdf(filename,format=kwargs['format'],encoding={'time':{'units':'days since 1900-01-01'}})
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncC

## Next Steps

QC of data (plot parameters with other instruments)
- be sure to updated the qc_status and the history